Сначала вам нужно будет установить и импортировать модуль Python H2O и класс H2OAutoML, как и любую другую библиотеку, и инициализировать H2O.

In [9]:
import h2o
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 33 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 7 days
H2O_cluster_name:,H2O_from_python_zhigu_4q5tvk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.758 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Затем нам нужно загрузить данные. Это можно сделать либо прямо в «H2OFrame» (закомментированный пример), либо обычным способом, чтобы мы могли закодировать целевую переменную, а затем преобразовать их в H2OFrame. 

Как и многие функции и объекты в H2O, H2OFrame схож с Pandas DataFrame, но имеет свои небольшие отличия и синтаксис. Например, H2O может самостоятельно обрабатывать категориальные признаки, поэтому выполнять этот шаг отдельно не требуется.

In [10]:
df = pd.read_csv("data/mushrooms.zip")
# df = h2o.import_file("mushrooms.csv", header =1)

labelEncoder = LabelEncoder()
for column in df.columns:
    df[column] = labelEncoder.fit_transform(df[column])
df = h2o.H2OFrame(df)
df = df.asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Получается, с использованием AutoML мы не будем проделывать эти шаги и сразу перейдём к обучению!

Подобно функциям в sklearn, мы можем создать разбиение на train и test, чтобы можно было проверить производительность модели на неизвестном наборе данных. 

In [11]:
# Разделяем данные на Train/Test/Validation. Train размером 70%, test и validation по 15% 
train_df, test_df = df.split_frame(ratios=[.7])

Затем нам нужно получить имена столбцов, чтобы передать их функции. 

Для AutoML необходимо указать несколько обязательных параметров: x, y и training_frame. Здесь вы также можете настроить значения max_runtime_sec и max_models. 

Max_runtime_sec — обязательный параметр, а max_model — необязательный. 
По умолчанию все непереданные параметры принимают значение NULL. 
Параметр x — это вектор признаков из training_frame. Если вы не хотите использовать все признаки из переданного кадра, вы можете установить его, передав его в x.
На этих данных будем использовать все признаки (кроме таргета) и установим max_runtime_sec на 10 минут (по умолчанию некоторые из моделей могут занять много времени). Запустим AutoML:

In [12]:
y = "class"
train_columns = train_df.columns
train_columns.remove(y)

aml_model = H2OAutoML(max_runtime_secs=500, seed = 1)
aml_model.train(x = train_columns, y = y, training_frame = train_df)

AutoML progress: |█
17:53:27.609: AutoML: XGBoost is not available; skipping it.
17:53:27.670: _train param, Dropping bad and constant columns: [veil-type]
17:53:32.274: _train param, Dropping bad and constant columns: [veil-type]

███████
17:54:10.920: _train param, Dropping unused columns: [veil-type]
17:54:13.947: _train param, Dropping bad and constant columns: [veil-type]
17:54:16.15: _train param, Dropping bad and constant columns: [veil-type]

███
17:54:36.283: _train param, Dropping bad and constant columns: [veil-type]

██
17:54:55.187: _train param, Dropping bad and constant columns: [veil-type]

███
17:55:17.498: _train param, Dropping unused columns: [veil-type]


17:55:19.823: _train param, Dropping unused columns: [veil-type]
17:55:23.213: _train param, Dropping bad and constant columns: [veil-type]

█
17:55:25.467: _train param, Dropping bad and constant columns: [veil-type]

██
17:55:41.650: _train param, Dropping bad and constant columns: [veil-type]
17:55:44.518: _tra

Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_1_20250704_175327


GLM Model: summary
    family    link    regularization               lambda_search                                                                    number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------  -------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  -----------------------------------------------
    binomial  logit   Ridge ( lambda = 1.947E-5 )  nlambda = 30, lambda.max = 19.466, lambda.min = 1.947E-5, lambda.1se = 1.947E-5  116                           116                            79                      AutoML_1_20250704_175327_training_py_3_sid_9953

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 1.6016796870223762e-05
RMSE: 0.004002099058022398
LogLoss: 0.0010720486230180236
AUC: 1.0
AUCPR: 1.0
Gini: 1.0
Null degrees of freedom: 5715
Residual degrees of freedom: 5599
Null deviance: 7916.193923354536
Residual deviance: 12.25565985834208
AIC: 246.25565985834209

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8891313578748679
       0     1     Error    Rate
-----  ----  ----  -------  ------------
0      2964  0     0        (0.0/2964.0)
1      0     2752  0        (0.0/2752.0)
Total  2964  2752  0        (0.0/5716.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.889131     1         192
max f2                       0.889131     1         192
max f0point5                 0.889131     1         192
max accuracy                 0.889131     1         192
max precision                0.999998     1         0
max recall                   0.889131     1         192
max specificity              0.999998     1         0
max absolute_mcc             0.889131     1         192
max min_per_class_accuracy   0.889131     1         192
max mean_per_class_accuracy  0.889131     1         192
max tns                      0.999998     2964      0
max fns                      0.999998     2340      0
max fps                      1.02864e-06  2964      399
max tps                      0.889131     2752      192
max tnr                      0.999998     1         0
max fnr                      0.999998     0.850291  0
max fpr                      1.02864e-06  1         399
max tpr                      0.889131     1         192

Gains/Lift Table: Avg response rate: 48.15 %, avg score: 48.15 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.010147                    1                  2.07703  2.07703            1                1            1                           1                   0.0210756       0.0210756                  107.703  107.703            0.0210756
2        0.020119                    0.999999           2.07703  2.07703            1                0.999999     1                           1                   0.0207122       0.0417878                  107.703  107.703            0.0417878
3        0.030091                    0.999999           2.07703  2.07703            1                0.999999     1                           0.999999            0.0207122       0.0625                     107.703  107.703            0.0625
4        

Мы задали значение параметра Max_runtime_sec ~10 минут, но также есть параметр, отвечающий не за время, а за максимальное количество исследованных моделей. Помимо этого, есть возможности дополнительной настройки процесса работы AutoML и множество дополнительных параметров, которые вы можете задать. 

Подробнее о параметрах можно прочитать в официальной документации.

После запуска и обучения H2OAutoML вы можете посмотреть, какие модели в нашем случае сработали лучше всего, и выбрать их для дальнейшего исследования.

In [13]:
leaderboard = aml_model.leaderboard
leaderboard.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GLM_1_AutoML_1_20250704_175327,1,0.00148887,1,0,0.00711334,5.05996e-05
DeepLearning_grid_1_AutoML_1_20250704_175327_model_1,1,0.00024599,1,0,0.00678681,4.60608e-05
StackedEnsemble_BestOfFamily_1_AutoML_1_20250704_175327,1,0.000847209,1,0,0.000847485,7.1823e-07
GBM_grid_1_AutoML_1_20250704_175327_model_1,1,1.54317e-13,1,0,1.13466e-11,1.28746e-22
GBM_grid_1_AutoML_1_20250704_175327_model_3,1,1.60042e-06,1,0,0.00010158,1.03185e-08
StackedEnsemble_AllModels_1_AutoML_1_20250704_175327,1,0.000677898,1,0,0.000679677,4.61961e-07
GBM_grid_1_AutoML_1_20250704_175327_model_7,1,3.14266e-14,1,0,1.77794e-12,3.16108e-24
StackedEnsemble_AllModels_3_AutoML_1_20250704_175327,1,0.000647937,1,0,0.000648353,4.20362e-07
GBM_grid_1_AutoML_1_20250704_175327_model_8,1,9.93531e-07,1,0,3.46542e-06,1.20092e-11
StackedEnsemble_AllModels_2_AutoML_1_20250704_175327,1,0.000667657,1,0,0.000668159,4.46436e-07


Давайте проверим результат предсказания на тестовых данных и посмотрим, не переобучилась ли модель:

In [15]:
prediction = aml_model.predict(test_df)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
